# 심층 Q-네트워크(DQN)로 CartPole 문제 풀기

**학습 목표:**
- 상태 공간이 매우 크거나 연속적인 문제에서 Q-테이블을 사용할 수 없는 한계를 이해하고, **딥러닝**을 결합한 DQN의 필요성을 학습합니다.
- 상태(state)를 입력받아 각 행동(action)의 Q-값을 출력하는 **신경망(Q-Network)**을 설계하고 구현합니다.
- 학습 안정성을 높이는 DQN의 핵심 기술인 **경험 리플레이(Experience Replay)**와 **타겟 네트워크(Target Network)**의 역할과 구현 방법을 배웁니다.
- **OpenAI Gym**의 **CartPole** 환경에서 DQN 에이전트가 막대의 균형을 오랫동안 유지하도록 학습시킵니다.

In [ ]:
import gym
import numpy as np
import random
from collections import deque
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt

### (1) DQN 에이전트 클래스 정의
DQN 알고리즘에 필요한 모든 구성 요소(Q-네트워크, 타겟 네트워크, 메모리, 하이퍼파라미터 등)와 기능(행동 선택, 경험 저장, 학습)을 하나의 클래스로 캡슐화하여 코드를 구조적으로 관리합니다.

In [ ]:
class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        
        # 경험 리플레이 메모리 (deque)
        self.memory = deque(maxlen=2000)
        
        # 하이퍼파라미터
        self.gamma = 0.95    # discount factor
        self.epsilon = 1.0   # exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.001
        
        # Q-Network와 Target Network 생성
        self.model = self._build_model()
        self.target_model = self._build_model()
        self.update_target_model()

    def _build_model(self):
        # Q-값을 근사할 신경망 모델
        model = Sequential([
            Dense(24, input_dim=self.state_size, activation='relu'),
            Dense(24, activation='relu'),
            Dense(self.action_size, activation='linear')
        ])
        model.compile(loss='mse', optimizer=Adam(learning_rate=self.learning_rate))
        return model

    def update_target_model(self):
        # 타겟 네트워크의 가중치를 메인 네트워크의 가중치로 업데이트
        self.target_model.set_weights(self.model.get_weights())

    def remember(self, state, action, reward, next_state, done):
        # 경험(s, a, r, s', done)을 메모리에 저장
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        # Epsilon-greedy 정책으로 행동 선택
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state, verbose=0)
        return np.argmax(act_values[0])

    def replay(self, batch_size):
        # 메모리에서 미니배치를 샘플링하여 학습 (경험 리플레이)
        if len(self.memory) < batch_size:
            return
        minibatch = random.sample(self.memory, batch_size)
        
        states = np.array([t[0][0] for t in minibatch])
        actions = np.array([t[1] for t in minibatch])
        rewards = np.array([t[2] for t in minibatch])
        next_states = np.array([t[3][0] for t in minibatch])
        dones = np.array([t[4] for t in minibatch])

        # 현재 상태에 대한 Q-값과 다음 상태에 대한 타겟 Q-값 예측
        q_values = self.model.predict(states, verbose=0)
        target_q_values = self.target_model.predict(next_states, verbose=0)

        # 벨만 방정식을 이용한 타겟값 계산
        targets = rewards + self.gamma * np.amax(target_q_values, axis=1) * (1 - dones)
        
        # 실제 선택했던 행동에 대한 Q-값만 타겟값으로 업데이트
        q_values[np.arange(batch_size), actions] = targets

        # 모델 학습
        self.model.fit(states, q_values, epochs=1, verbose=0)
        
        # Epsilon 감소
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

### (2) DQN 학습 실행
CartPole-v1 환경에서 에이전트를 생성하고, 지정된 에피소드 수만큼 학습을 진행합니다. 각 에피소드는 막대가 쓰러지거나 최대 타임스텝(500)에 도달하면 종료됩니다.

In [ ]:
env = gym.make('CartPole-v1')
state_size = env.observation_space.shape[0]
action_size = env.action_space.n
agent = DQNAgent(state_size, action_size)

episodes = 500
batch_size = 64
update_target_every = 10
scores = []

for e in range(episodes):
    state = env.reset()
    state = np.reshape(state, [1, state_size])
    done = False
    time_steps = 0
    
    while not done:
        time_steps += 1
        action = agent.act(state)
        next_state, reward, done, _ = env.step(action)
        reward = reward if not done or time_steps == 500 else -10 # 실패 시 음수 보상
        next_state = np.reshape(next_state, [1, state_size])
        
        agent.remember(state, action, reward, next_state, done)
        state = next_state
        
        if done:
            scores.append(time_steps)
            print(f"episode: {e+1}/{episodes}, score: {time_steps}, e: {agent.epsilon:.2}")
            break
    
    agent.replay(batch_size)
    
    if e % update_target_every == 0:
        agent.update_target_model()
        
env.close()

### (3) 학습 결과 시각화

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(scores)
plt.title('DQN Agent Training on CartPole-v1')
plt.xlabel('Episode')
plt.ylabel('Score (Time Steps)')
plt.grid(True)
plt.show()
print("학습이 진행됨에 따라 에이전트가 더 오랫동안 균형을 유지하는 것을 볼 수 있습니다.")